In [1]:
import os
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, skewnorm

def analyze_stock_data(tickers, start_date, end_date, trading_days, risk_free_rate):
    # Download S&P 500 data
    sp500 = yf.download('^GSPC', start=start_date, end=end_date)
    sp500['Daily Return'] = sp500['Close'].pct_change().dropna()

    # Initialize a list to store metrics
    metrics = []

    for company, ticker in tickers.items():
        try:
            # Download stock data
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            if stock_data.empty:
                raise ValueError(f"No data for {company} ({ticker})")

            # Calculate daily return
            stock_data['Daily Return'] = stock_data['Close'].pct_change().dropna()

            # Calculate mean return, volatility, and Sharpe ratio
            mean_return = stock_data['Daily Return'].mean() * trading_days
            volatility = stock_data['Daily Return'].std(ddof=1) * np.sqrt(252)
            sharpe_ratio = (mean_return - risk_free_rate) / volatility
            beta = stock_data['Daily Return'].cov(sp500['Daily Return']) / sp500['Daily Return'].var(ddof=1)

            # Calculate skewness and kurtosis (handle NaN issues by ensuring enough data)
            daily_returns_valid = stock_data['Daily Return'].dropna()
            skewness = skew(daily_returns_valid)
            kurt = kurtosis(daily_returns_valid)

            # 1. Empirical probability of negative returns
            prob_negative_empirical = (daily_returns_valid < 0).mean()
            mu = daily_returns_valid.mean()
            sigma = daily_returns_valid.std()
            prob_negative_skewnorm = skewnorm.cdf(0, skewness, loc=mu, scale=sigma)

            # Append calculated metrics
            metrics.append([
                company, 
                mean_return, 
                volatility, 
                sharpe_ratio, 
                skewness, 
                kurt, 
                beta, 
                prob_negative_empirical, 
                prob_negative_skewnorm
            ])

        except Exception as e:
            print(f"Error with {company} ({ticker}): {e}")
            continue

    # Convert metrics list to a DataFrame
    columns = [
        'Company', 'Mean Return', 'Volatility', 'Sharpe Ratio', 'Skewness', 
        'Kurtosis', 'Beta', 'Prob Negative (Empirical)', 'Prob Negative (Skewnorm)'
    ]
    data_frame = pd.DataFrame(metrics, columns=columns)

    
    data_frame.sort_values(by='Sharpe Ratio', ascending=False, inplace=True)        # Sort by 'Sharpe Ratio'
    os.makedirs("Excel", exist_ok=True)                                             # Create Excel folder if it doesn't exist
    file_path = os.path.join("Excel", f'rf {risk_free_rate} days {trading_days} from {start_date} to {end_date}.xlsx')    
    data_frame.to_excel(file_path, index=False)                                     # Save to Excel

    return data_frame

In [7]:
# Read stock from file
with open('stocks.txt', 'r') as file:
    tickers = {line.strip(): line.strip() for line in file}

start_date = '2024-10-07'
end_date = '2024-11-08'
risk_free_rate = 0.008
days = 25

# Analyze stock data
data_frame = analyze_stock_data(tickers, start_date, end_date, days, risk_free_rate)
print(data_frame)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  0 of 0 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Error with  (): No objects to concatenate
  Company  Mean Return  Volatility  Sharpe Ratio  Skewness  Kurtosis  \
0   ^GSPC     0.052547    0.134050      0.332318  0.217008  1.837038   
1    ^MXX     0.006293    0.116576     -0.014639  0.588521 -0.161540   

       Beta  Prob Negative (Empirical)  Prob Negative (Skewnorm)  
0  1.000000                   0.434783                  0.335800  
1  0.310503                   0.565217                  0.317116  
